In [74]:
# coding: utf-8
# SNLP - SoSe 2020 - ASSINGMENT VI

import math
import re
import operator
from collections import defaultdict, Counter
from nltk import ngrams
import pandas as pd

In [75]:
with open('English_train.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()

with open('English_test.txt', 'r', encoding='utf8') as f:
    test_lines = f.readlines()

In [76]:
def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.sub(r'[^a-z\s\n\-]', '', text.lower()).split()
    # return re.findall('[a-z]+', text.lower()).split()

In [77]:
def word_ngrams(sent, n):
    """Givne a sent as str return n-grams as a list of tuple"""
    
    tp=()
    listtp= []
    token = tokenize(sent)
    #if it is not 1-gram append <s> to the begining and </s> at the end
    if(n!=1):
        token.insert(0,"<s>")
        token.append("</s>")
    
    #generate n-grams using the zip function in python
    ngrams = zip(*[token[i:] for i in range(n)])
    
    #loop through the n-grams generated and create list of tuples
    for ngram in ngrams:
        listtp.append(ngram)
        
    return listtp


In [78]:
#We declare this method to generate the n-grams assisting the 
def getGram(lines, n):
  bigramst=[]
  if type(lines) == list:
    for sentences in lines:
      bg=word_ngrams(sentences, n)
      bigramst+=bg
  else:
    bigramst = word_ngrams(lines, n)
  return bigramst

In [73]:
class ngram_LM:
    """A class to represent a language model."""

    def __init__(self, n, ngram_counts, vocab, unk=False):
        """"Make a n-gram language model, given a vocab and
            data structure for n-gram counts."""
        
        self.n = n 
        
        self.vocab = vocab
        
        self.V = len(vocab)

        self.ngram_counts = ngram_counts

  # YOUR CODE HERE
  # START BY MAKING THE RIGHT COUNTS FOR THIS PARTICULAR self.n         
       

    #@functools.lru_cache(maxsize=None) #Caches function calls with the same value, just as a test on how this works https://stackoverflow.com/questions/1988804/what-is-memoization-and-how-can-i-use-it-in-python
    def estimate_smoothed_prob(self, history, word, d=0.7):
        """Estimate probability of a word given a history with absolute discounting smoothing."""
        # YOUR CODE HERE
        alpha = 0.5
        if self.n == 1:
            return (alpha + self.ngram_counts[word])/(alpha*self.V + len(v_unigrams)) #sum(self.ngram_counts.values()))
            
        else:
            return (alpha + self.ngram_counts[(history, word)])/(alpha*self.V + v_unigrams[(history,)])
        


    def logP(self, history, word):
        """Return base-2 log probablity."""
        # YOUR CODE HERE
        return math.log(self.estimate_smoothed_prob(history, word, 0.5), 2)

    # for smoothed probability:
    def test_LM(self):
        """Test whether or not the smoothed probability mass sums up to one."""
        
        precision = 10**-8
        
        # as for unigram we need to take histories as empty string.
        if self.n == 1:
                 
            P_sum = sum(self.estimate_smoothed_prob('', w) for w in self.vocab)
            
            if abs(1.0 - P_sum) < precision:
              print( 'TEST SUCCESSFUL!')
                 
        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']
            
            for h in histories:
                P_sum = sum(self.estimate_smoothed_prob(h, w) for w in self.vocab)
                
                if abs(1.0 - P_sum) < precision:
                  print( 'TEST SUCCESSFUL!')
        
        print('TEST SUCCESSFUL!')
        return precision

    def estimate_prob(self, history, word):
        """Estimate probability of a word given a history."""
        probability=0
        denominator=0
        if(self.n==1):
          #count the number of times word exist
          probability=self.ngram_counts[word]/len(v_unigrams)
        else:
          #count the number of times words exist with history
            numerator=self.ngram_counts[(history,word)]
            denominator= ugCount[(history,)]
            probability=numerator/denominator
        return probability
      
    def test_Normal_LM(self):
        """Test whether or not the probability mass sums up to one."""
        
        precision = 10**-8
                 
        if self.n == 1:
                 
            P_sum = sum(self.estimate_prob('', w) for w in self.vocab)
            
            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'
                 
        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']
                 
            for h in histories:

                P_sum = sum(self.estimate_prob(h, w) for w in self.vocab)
                
                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history' + h
                     
        print('TEST SUCCESSFUL!')
        

# ADD YOUR CODE TO COLLECT COUNTS AND CONSTRUCT VOCAB
v_unigrams = getGram(lines, 1)
v_bigrams = getGram(lines, 2)
unigram_counts=Counter(v_unigrams)
bigram_counts=Counter(v_bigrams)
# print(unigram_counts['the',])
print(unigram_counts)
print(bigram_counts)

tv_unigrams = getGram(test_lines, 1)
tv_bigrams = getGram(test_lines, 2)
testUnigrams=Counter(tv_unigrams)
testBigrams=Counter(tv_bigrams)

# ONCE YOU HAVE N-GRAN COUNTS AND VOCAB, 
# YOU CAN BUILD LM OBJECTS AS ... 
unigram_LM = ngram_LM(1, unigram_counts, v_unigrams)
bigram_LM = ngram_LM(2, bigram_counts, v_bigrams)

test_bigram_LM = ngram_LM(2, testBigrams, tv_bigrams)


def retrieve_lang_models():
    return unigram_LM, bigram_LM

print("\n\nTest Unigram LM")
unigram_LM.test_LM()
print("\nTest Bigram LM")
bigram_LM.test_LM()



Counter({('the',): 3493, ('to',): 3156, ('of',): 2917, ('and',): 2837, ('her',): 1723, ('a',): 1584, ('i',): 1510, ('in',): 1503, ('was',): 1376, ('she',): 1306, ('that',): 1223, ('it',): 1162, ('not',): 1127, ('he',): 1024, ('his',): 994, ('be',): 930, ('you',): 927, ('had',): 912, ('as',): 904, ('with',): 838, ('for',): 803, ('but',): 746, ('is',): 711, ('mr',): 670, ('have',): 642, ('at',): 605, ('him',): 567, ('on',): 542, ('my',): 538, ('by',): 511, ('they',): 490, ('all',): 472, ('elizabeth',): 459, ('were',): 451, ('so',): 432, ('which',): 421, ('could',): 410, ('very',): 402, ('been',): 391, ('from',): 381, ('no',): 370, ('this',): 355, ('their',): 354, ('them',): 353, ('what',): 351, ('would',): 339, ('your',): 327, ('will',): 312, ('such',): 303, ('said',): 303, ('darcy',): 300, ('me',): 298, ('when',): 285, ('an',): 284, ('there',): 282, ('are',): 270, ('mrs',): 269, ('do',): 268, ('much',): 266, ('if',): 257, ('am',): 255, ('miss',): 248, ('more',): 241, ('must',): 228, ('w

TypeError: ignored

In [79]:
#perplexity


def compute_perplexity(interp = True):
    sumlog = 0
    for bigram in testBigrams:
        word1 = testBigrams[0]
        word2 = testBigrams[1]
        if interp:
            bigram_prob = bigram_LM.estimate_smoothed_prob(word1, word2)
        else:
            bigram_prob = bigram_LM.estimate_prob(word1, word2)
        log_bigram_prob = math.log(bigram_prob, 2)
        sumlog = sumlog + log_bigram_prob
    perplexity = pow(2, (-1) * sumlog/len(testBigrams))  
    return perplexity          
    
print("Perplexity with Interpolation: ", compute_perplexity(True))
print("Perplexity no Interpolation: ", compute_perplexity(False))

TypeError: ignored